# Dependencies

Change the following to code cells and run them to set-up the environment

# Searching Vision for Specific Skills

In [1]:
import requests
import json

In [2]:
# Used when loading JSON to give access to keys as attributes of the object
class Struct(dict):
    def __getattr__(self, name):
        return self[name]

    def __setattr__(self, name, value):
        self[name] = value

    def __delattr__(self, name):
        del self[name]

In [18]:
request_url = "https://mbisearchemployeednc.azurewebsites.net/?cmd=search&employee=fake.name@mbakerintl.com&subdata=skills,compskills,liccerts&criteria=where%20(service%20like%20%27%25Rail%25%27%20or%20feature%20like%20%27%25Rail%25%27)"

In [19]:
employee_url = "https://mbisearchemployeednc.azurewebsites.net/?cmd=user&employee=dane.parks@mbakerintl.com&subdata=education,skills,compskills,liccerts"

In [20]:
mbi_employee_resources = requests.get(request_url, headers={'origin': 'https://mbakerintl.sharepoint.com/'})

In [21]:
search_results = json.loads(mbi_employee_resources.content, object_hook=Struct)

In [22]:
search_results[10].WorkPhone

'412-918-4028'

In [23]:
search_results[0]

{'Employee': '809056',
 'Email': 'Johann.Aakre@mbakerintl.com',
 'LastName': 'Aakre',
 'FirstName': 'Johann',
 'MiddleInitial': 'F.',
 'Region': 'Great Lakes',
 'Suffix': 'P.E., S.E.',
 'TotalYearsExperience': '21',
 'YearsAtMBI': '6',
 'MobilePhone': '312-363-8778',
 'WorkPhone': '312-575-3907',
 'Org': '495.51.19.51',
 'Department': 'Bridges-Practice Resource-Chicago, IL',
 'ManagerName': 'Aaron Stover',
 'ManagerEmail': 'astover@mbakerintl.com',
 'BakerOffice': 'Chicago, IL',
 'BakerTitle': 'Regional Operations Lead - Bridge',
 'ExecutiveTitle': 'Associate Vice President',
 'Practice': 'Bridge',
 'IndirectReportsCount': '0',
 'DirectReportsCount': '0',
 'TotalReportsCount': '0',
 'FlattenedLicensesCertifications': 'Professional Engineer (Iowa)<BR>Professional Engineer (Illinois)<BR>Professional Engineer (Michigan)<BR>Structural Engineer (Illinois)',
 'FlattenedSkills': 'Construction Manager/General Contractor (CM/GC) (0 - Unknown)<BR>Abutment : Design (2 - Intermediate)<BR>Bridge - 

In [17]:
for Employee in search_results[:10]:
    print(f"Name: {Employee.FirstName} {Employee.LastName}")
    print(f"Headquarters: {Employee.BakerOffice}")
    print(f"Reigon: {Employee.Region}")
    print(f"Departmetn: {Employee.Department}\n\n")

Name: Johann Aakre
Headquarters: Chicago, IL
Reigon: Great Lakes
Departmetn: Bridges-Practice Resource-Chicago, IL


Name: Steven Ackerman
Headquarters: Moon Township, PA
Reigon: Great Lakes
Departmetn: ArchPlanBuild-Practice Resource-Fed Pr Mgmt


Name: Timothy Ainsley II
Headquarters: Harrisburg, PA
Reigon: Great Lakes
Departmetn: Bridges-Practice Resource-Harrisburg, PA


Name: Wayne Airgood
Headquarters: Moon Township, PA
Reigon: Great Lakes
Departmetn: Structural-Practice Resource-Structural


Name: Akin Akinduro
Headquarters: Nashville, TN
Reigon: Southern
Departmetn: Bridges-Practice Resource-Nashville, TN


Name: Kayla Albertson
Headquarters: Tallahassee, FL
Reigon: Southern
Departmetn: Hwy/Bridge-Practice Resource-Tallahassee, FL


Name: Constantin Aldea
Headquarters: Hamilton, NJ
Reigon: Northeast
Departmetn: Bridges-Practice Resource-Hamilton, NJ


Name: Mir Ali
Headquarters: Little Rock, AR
Reigon: Southern
Departmetn: Bridges-Practice Resource-Little Rock, AR


Name: Ainsl

Geocoding the address

In [ ]:
from geopy.geocoders import Nominatim
loc = Nominatim(user_agent="GetLoc")

In [ ]:
getLoc = loc.geocode(search_results[0].BakerOffice)

In [ ]:
getLoc.latitude, getLoc.longitude

# Associating Employees with an Office Utilizing 'Nearest Neighbor'

In [ ]:
import pandas as pd

employees = pd.DataFrame()

In [ ]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [ ]:
# Filtering skills table to get results related to rail
[x for x in search_results[0]['Skills'] if "rail" in x['Feature'].lower()]

In [ ]:
skills = [x for x in search_results[0]['Skills'] if 'rail' in x['Feature'].lower()]

In [ ]:
skills[0]['Feature']

In [ ]:
from geopy.geocoders import Nominatim

def convert_search_results_to_df(search_results, search_term):
    """
    This function takes a long time to run because of the geocoding aspect, ideally
    for a production application, commonly searched terms would be generated and cached
    so that it wasn't constantly getting rebuilt.
    """
    loc = Nominatim(user_agent="GetLoc")
    employees = []
    i = 1  # Only used to limit the print statements to every 10th result
    
    for employee in search_results:
        if i % 10 == 0:
            print(f"{i} Employees added to DataFrame")
            
        # Builds out the list of dictionaries that employees skills are stored in
        skills = [x for x in employee['Skills'] if search_term in x['Feature'].lower()]
        # Pandas breaks if you pass an empty string as a dictionary value, so filter those
        skills = [{k: v for k, v in d.items() if v} for d in skills]
            
        try:  # The Geocoder isn't always able to find the location
            coordinates = loc.geocode(employee.BakerOffice, timeout=10)
            
            employees.append(
                {
                    'Name': f"{employee.FirstName} {employee.LastName}",
                    'Email': employee.Email,
                    'YoE': employee.TotalYearsExperience,
                    'Manager': employee.ManagerName,
                    'Practice': employee.Practice,
                    'PhoneOff': employee.WorkPhone,
                    'PhoneMob': employee.MobilePhone,
                    'Headquarters': f"{employee.BakerOffice}",
                    'Reigon':  f"{employee.Region}",
                    "Department": f"{employee.Department}",
                    'Latitude': coordinates.latitude,
                    'Longitude': coordinates.longitude,
                    'Skills': skills
                }
            )
        except:  # If it doesn't, use None for the location
            coordinates = loc.geocode('')
            print(f"{bcolors.WARNING}{i-1}: {employee.BakerOffice} {coordinates}{bcolors.ENDC}")

            employees.append(
                {
                    'Name': f"{employee.FirstName} {employee.LastName}",
                    'Email': employee.Email,
                    'YoE': employee.TotalYearsExperience,
                    'Manager': employee.ManagerName,
                    'Practice': employee.Practice,
                    'PhoneOff': employee.WorkPhone,
                    'PhoneMob': employee.MobilePhone,
                    'Reigon':  f"{employee.Region}",
                    "Department": f"{employee.Department}",
                    'Latitude': '',
                    'Longitude': '',
                    'Skills': skills
                }
            )
    
        i += 1
    
    employees_df = pd.DataFrame(employees)

    return employees_df

In [ ]:
employees_df = convert_search_results_to_df(search_results, 'rail')

In [ ]:
employees_df

In [ ]:
import geopandas as gpd

In [ ]:
gdf = gpd.GeoDataFrame(employees_df, geometry=gpd.points_from_xy(employees_df.Longitude, employees_df.Latitude), crs="EPSG:4326")

In [ ]:
# We now have a geometry column
gdf

In [ ]:
gdf.explore()

using 'Marker Clusters' since multiple datapoints are occuring at the same location.

In [ ]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=[37, -87], zoom_start=5)
marker_cluster = MarkerCluster().add_to(m)

# Iterate through list and add a marker for each volcano, color-coded by its type.
for i, employee in gdf.iterrows():
    try:
        skills_string = '<br>'.join([f"{x['Feature']}: {x['Level']}" for x in employee['Skills']])
        summary_string = f"{employee['Name']}<br>{employee['Headquarters']}<br>{employee['Department']}<br><br>{skills_string}"
        
        # assign a color marker for the type of volcano, Strato being the most common
        if max(d['Level'] for d in gdf.iloc[i]['Skills']) == '4 - Expert':
            type_color = "green"
        elif max(d['Level'] for d in gdf.iloc[i]['Skills']) == '3 - Advanced':
            type_color = "blue"
        elif max(d['Level'] for d in gdf.iloc[i]['Skills']) == '2 - Intermediate':
            type_color = "orange"
        elif max(d['Level'] for d in gdf.iloc[i]['Skills']) == '1 - Basic':
            type_color = "red"
        else:
            type_color = "purple"

        folium.Marker(
        location=[gdf.iloc[i]['Latitude'], gdf.iloc[i]['Longitude']],
        popup=folium.Popup(summary_string, min_width=300, max_width=300),
        icon=folium.Icon(color=type_color, icon="ok-sign"),
        ).add_to(marker_cluster)
        
    except:
        print(f"Employee: {gdf.iloc[i]['Name']} Skills List: {gdf.iloc[i]['Skills']}")

In [ ]:
m.save('test_map.html')

In [ ]:
# Set filepath
fp = "res/MB Offices/Michael_Baker_International_Offices.shp"

# Read file using gpd.read_file()
office_data = gpd.read_file(fp)

In [ ]:
def add_geodataframe_with_custom_icons(folium_map, gdf):
    # Iterate through the GeoDataFrame rows
    for _, row in gdf.iterrows():
        # Extract the coordinates for each row from the Point geometry
        coords = [row['geometry'].y, row['geometry'].x]
        # Create a custom marker with an icon
        folium.Marker(
            location=coords,
            popup=row['name'],
            icon=folium.Icon(
                color='black',
                prefix='fa',
                icon='building'
            )
        ).add_to(folium_map)

    return folium_map

In [ ]:
m3 = add_geodataframe_with_custom_icons(m, office_data)

In [ ]:
m

In [ ]:
m2

In [ ]:
input('breakpoint')

# Finding Incomplete Data and Notifying Employees

## Automated Emailer

Doesn't work, likely a block on IT's end, takes a long time and goes to junk mail

In [ ]:
import json
 
# Opening JSON file - Application Keys are stored here so you can't see my passwords
f = open('../secrets.json')
data = json.load(f)

# These values would have to be replaced to successfully run this script, the email value is the "from" email you'll be using
gmail_account = data['GMAIL_ACCOUNT']  # danecodetest
gmail_email = data['GMAIL_EMAIL']      # danecodetest@gmail.com
gmail_pass = data['GMAIL_PASS']        # Application key for automating a Gmail account
 
# Closing file
f.close()

In [ ]:
import smtplib
import string

def smtp_gmail(user_email, project_list):
    username = gmail_account    
    password = gmail_pass
    email_from = gmail_email
    email_to = user_email
    
    smtp_server = "smtp.gmail.com:587"
    print(f"Attempting to Use SMTP Server: {smtp_server}")
    
    print(f"Sending Notification to {email_to}\n")
    
    with open("res/email_demo.txt", 'r') as email_text:
        x = email_text.read()
    
    projects_string = '\n\n'.join((str(x) for x in project_list))
    
    email_body = "\r\n".join((
        f"From: {email_from}",
        f"To:{email_to}",
        "Subject: Missing Values in Vision",
        "",
        f"{projects_string}",
        ), 
    )
    
    print(f"Message Preview:\n\n{email_body}")
    
    # The rest of this is commented out to avoid spamming
    # server = smtplib.SMTP(smtp_server)
    print("\nstarted server")
    
    # server.starttls()
    print("started tls")
    
    # server.login(username, password)
    print("logged into gmail")
    
    # server.sendmail(email_from, email_to, email_body)
    print("sent email")
    
    # server.quit()

In [ ]:
try:
    smtp_gmail("dane.parks@mbakerintl.com", [1, 2, 3, 4])
except:
    print('Sending Email Failed')

# Available Skill Codes

In [ ]:
3D Parametric Model	 	ISO 14001 – Environmental Management Systems
Above-ground Storage Tanks	 	ISO 14040 – Life Cycle Assessments
Abutment	 	ISO 31000 – Risk Management
Access Roads	 	ISO 55000 – Asset Management
Activity Overview Plans	 	ISO 9000 – Quality Management
Activity Planning & Management Models	 	ISO 9001 – Quality Management
Affordable Housing	 	ITS - Roadway Sensors
Air	 	ITS - RWIS
Air Carrier Airport	 	Joint-Use Airport
Air Content	 	Judicial and Courtroom Facilities
Air Installation Compatible Use Zones	 	K-Tag
Air Operations Area	 	Laboratories/Research Facilities
Air Terminal	 	Lagoons
Air Traffic	 	Landslides
Aircraft Fuel Facilities	 	Lane Management
Aircraft Rescue/Firefighting Facility	 	Lead-Based Paint
Airfield Electric Field Vault	 	Legislation
Airfield Lighting	 	Libraries/Museums/Galleries
Airfield Signage	 	Light Rail Transit
Airport - Airside	 	Lighting (Exteriors/Street/Memorials/Athletic Fields/etc.)
Airport - Landside	 	Lighting (Interior/Displays/Theatres/etc.)
Airport Layout Plan	 	Lighting Vault
Airport Master Plan	 	Liners
Airspace Hazards	 	Local Agency Formation Commissions (LAFCo)
Airspace Obstructions	 	Local Roads
Alliance for Tolling Interoperability (ATI)	 	Loop Detector
Amphibian Species	 	Low & Moderate Income Program
Amusement Parks	 	Magazines/Ammo Storage
Animal Facilities/Animal Shelters	 	Maintenance Facilities
Annexation	 	Maintenance On-line Management System (MOMS)
Apron/Ramp	 	Maintenance/Protection of Traffic (MPT)
Arctic Facilities	 	Mammalian Species and Signs
Area Development Plans	 	Managed Lanes
Area Plan	 	Manual
ASIP	 	Manual Toll
Asphaltic Concrete	 	Mapping Location/Addressing Systems
Assessment District	 	Marinas
Asset Management, A-GIS (Airport GIS)	 	Marine Infrastructure
Auditoriums/Theatres	 	Mass Timber Construction
Augmented Reality (AR)	 	Master Plan
Automated Storage and Retrieval Systems (ASRS)	 	Materials Handling Systems/Conveyors/Sorters
Automated Vehicle Identification (AVI)	 	Mello Roos CFD
Automatic Clearing House (ACH)	 	Microgrid
Automatic Number Plate Recognition (ANPR)	 	Military Airfield
Automatic/Automated Vehicle Classification (AVC)	 	Military Building
Autonomous Vehicle	 	Military Design Standards
Back Office Systems (BOS)	 	Military Facilities, Armory
Baggage Handling System	 	Military Housing/Barracks
Base Facility Requirements	 	Military Training Facilities
Basic Facility Requirement (BFR)	 	Mine Seals
Biological Resources	 	Mine Subsidence
Bird Species	 	Mines
Bituminous/Asphalt Pavement	 	Missile Facilities (Silos/Fuels/Transport)
Bridge - Arch	 	Mission Sustainment Plan
Bridge - Cable Stayed	 	MnPASS
Bridge - Girders/Beams	 	Mobile Application
Bridge - Historic	 	Mobile Home Development
Bridge - Movable	 	Modular Systems/Pre-fab Structures or Components
Bridge - Pedestrian	 	Mold
Bridge - Rail	 	Monitoring Well
Bridge - Segmental Concrete	 	Multi-Family/Apartments/Condominiums
Bridge - Spliced Bulb Tee	 	Multimedia/Audio-Visual Aids
Bridge - Suspension	 	Municipal Wastewater Programs
Bridge - Truss	 	Municipal Water Programs
Bridge – Underwater	 	National Bridge Inspection Standards (NBIS)
Bridge Foundations	 	National Environmental Policy Act (NEPA)
Bridge Hydraulics	 	NAVAIDs
Bridge over rail	 	Naval Architecture/Off-Shore Platforms
Bridges	 	Naval Station
Brownfield	 	Navigation Structures/Locks
Building Foundations	 	NEPA
Buried Structure	 	New Energy Sources
Bus Rapid Transit (BRT)	 	Next Generation 9-1-1 (NG9-1-1)
Bus Station/Shelter, Transit Center	 	Nexus Study
Bus Transit	 	Nodal Development Plans
Bus Transit and Paratransit	 	Noise
Busways	 	Noise Wall
California Environmental Quality Act (CEQA)	 	NORPASS
Caltrans Standards	 	Nuclear Facilities/Nuclear Shielding
Cantilever	 	Observatory
Capacity Improvement	 	Obstruction
Capital Improvement Plan	 	Office Building/Complexes/Industrial Parks
Casino	 	Oil and Hazardous Substance (OHS) Plans
Cast-in-Place Post-Tensioned Concrete	 	On-Call / Extension of Staff
CAT I Runway	 	Ordnances/Munitions/Special Weapons
CAT II and III Runway	 	Outfall
CAT II Runway	 	Parking Garages/Decks/Lots
CAT III Runway	 	Parks
Catenary	 	Passenger Boarding Bridge
Cemeteries	 	Payment Card Industry (PCI)
Child Care/Development Facilities	 	Pedestrian Grade Separations
Churches/Chapels	 	Pedestrian/Bike Structure
City Streets	 	Perfluorooctanoic Acid/Pulyfluorinated Substance
Civic Buildings, Community Centers	 	Perimeter Security Plan
Civil/Pavement	 	Petroleum and Fuel Storage and Distribution Facilities
Civil/Site	 	Petroleum Products
Clean Air Act	 	Petroleum Storage Tank (PST)
Clean Water Act	 	Pikepass
Climate Action Plan	 	Pile Footing
Climate Change	 	Pipelines
Climate Resilience Plan	 	Planetarium
Closed Circuit Television (CCTV)	 	Plans
CMMS (Computerized Maintenance Management System)	 	Plastic/Polymer
Coastal	 	PNAs/PAHs
Coastal Barriers and Zones	 	Pneumatic Structures/Air Support Buildings
Coastal Infrastructure	 	Point of Access Report
Combined Sewer Overflows (CSO)/Sanitary Sewer Overflows (SSO)	 	Port
Combined Sewer Systems	 	Portland Cement Concrete (PCC)
Command Facility	 	Post-Tension Concrete
Commercial Building, Shopping Centers	 	Postal Facilities
Commercial Service Airport	 	Powerhouse Building/Power Generation, Transmission, Distribution
Commercial Vehicle Operations	 	Precast Post-Tensioned Concrete
Communication Facility	 	Preliminary Design Authority (PDA)
Communications Audit/Plan	 	Preliminary Hazards List
Communications Systems/TV/Microwave	 	Prestressed Concrete
Community Development Block Grant (CDBG)	 	Prisons and Correctional Facilities
Commuter and Inner City Passenger Rail	 	Public Finance
Complete Streets	 	Public Finance Community Facility Districts and Assessment Districts
Comprehensive Neighborhood Plan	 	Public Finance Water and Sewer Rate Studies and Capital Improvement Plans
Computer Facilities	 	Public Safety Facilities
Concrete	 	Public-Private Partnership (PPP)
Concrete Pavement	 	Pump Station / Lift Station
Connected Vehicle	 	Radar/Sonar/Radio and Radar Telescopes
Conservation Plan	 	Radio Frequency Systems and Shieldings
Consolidated Plan	 	Radioactive/Mixed Wastes
Construction Cost Estimates	 	Radon
Construction Manager/General Contractor (CM/GC)	 	Rail Crossings
Control Centers	 	Rail Maintenance Facilities
Convention Center	 	Rail Stations/Passenger Facilities
Crisis and Consequence Management Plan (CCMP)	 	Rail Transit
Cryogenic Facilities	 	Rail Yard
CSO Long-term Control Plans	 	Rail Yard Support Buildings
Culvert	 	Rail Yard/Yard Support Buildings
Customer Service Center (CSC)	 	Railroad and Rapid Transit
Dams	 	Railway Tunnels
Declared Distances	 	Ramp Toll Facilities
Dedicated Lanes	 	Range Air Installation Compatible Use Zones
Dedicated Short Range Communication (DSRC)	 	Readiness Center
Deep Foundations	 	Recreational Facilities
Deicing Equipment	 	Recycled Materials
Deicing Facilities	 	Refrigeration Plants/Systems
Deicing System	 	Regional Integration Plans
Development Impact Fee Study	 	Reinforced Concrete Superstructures (Box, Beams, Slabs)
Dike/Levee	 	Reinforced Earth Wall
Dining Halls/Clubs/Restaurants	 	Remedial Action
Displaced Runway Threshold	 	Renderings
Diversion Chambers	 	Rental Car Facility
DNAPLS	 	Reptilian Species
Dormitory	 	Reserves Facility
Drilled Shaft	 	Reservoir Operation Plan
E-ZPass	 	Residential
Educational Facilities/Classrooms/Training Facilities	 	Resort
Educational Facilities–K-12	 	Retaining Wall
Educational Facilities–Universities	 	Retirement Communities
EGIS (Enterprise GIS System)	 	Revenue
Electric Magnetic Fields	 	Revitalization Plan/Downtown Plan
Electronic Funds Transfer	 	Rivers/Canals/Waterways
Electronic Laboratory	 	Roadside Toll Collection System (RTCS)
Electronic Toll Collection (ETC)	 	Roadway Shoulder
Electronics	 	Roundabout
Elevators/Escalators/People-Movers	 	RPLANS
eManual	 	Runway
Embankment	 	Runway Protection Zone
Embassies/Chanceries	 	Runway Safety Area
Emergency Action Plan	 	Runway Threshold
Emergency Planning and Community Right ot Know Act (EPCRA)	 	Safe Drinking Water Act
Encroachment Action Plans	 	Salt Dome
Energy and Greenhouse Gas Emissions	 	Sanitary Sewers
Entry/Access Control Point	 	School Crossing
Environmental Areas	 	SCIF Facility
EPASS	 	Section 106 Resources
ePRISMS	 	Section Maintenance Facility
Esri	 	Security Checkpoints
Exclusive Bus Lane	 	Security Fence
Express Toll Lane	 	Security Master Plan
Facility Master Plan	 	Security Scanning Equipment
Facility Response Plans (FRP)	 	Security Systems/Surveillance Systems
Facility Space Optimization Plan	 	Segmental Concrete
Fallout Shelters/Blast-Resistant Structure	 	Sensitive Compartmented Information Facility (SCIF Facility)
FAR Part 77 Surfaces	 	Service Area/Rest Area
Farmlands	 	Sewage Collection, Treatment and Disposal Facilities
Fast Track Schedule	 	Shale Formation
FasTrak	 	Sheet Pile Wall
Fee Study	 	Shipyard
Ferry Terminals and Transfer Bridges	 	Sidewalk/Curb/Gutter
FHWA Manual of Uniform Traffic Control Devices	 	Sign Structure
Fiber Reinforced Polymer (FRP)	 	Simple Network Management Protocol (SNMP)
Field House; Gym; Stadium; Arena	 	Single-Point Urban Interchange (SPUI)
Field Weather	 	Slurry Impoundment
Field Weather - Humidity	 	Soldier Pile Wall
Field Weather - Rain	 	Solid and Hazardous Waste
Field Weather - Temperature	 	Sound Barrier
Field Weather - Wind	 	Source Water
Financial Establishments/Banks	 	Spatial Data Standards for Facilities, Infrastructure, and Environment (SDSFIE)
Fire Alarm	 	Special Environments/Clean Rooms
Fire Station	 	Special Structures
Firing Range	 	Specialty Intersection
Fisheries/Fish Ladders	 	Sphere of Influence Update/Amendment
Fitness Center	 	Spill Prevention Countermeasure Control (SPCC) Plans
Flood Control Infrastructure	 	Spread Footings
Floodplain	 	Standby (Emergency) Power Supply
Flow Control Facilities	 	State Historic Preservation Officer (SHPO)
FMS (Facility Maintenance System)	 	State Police
Forestry/Forest Products	 	State Police - Contract
Foundation	 	State Police - Facilities
Freight Handling (Air Cargo Facilities)	 	State Police Barracks
Freight Rail	 	Steel
Freight Terminal Facilities	 	Stone
Frozen Soil	 	Storm Sewers
Fuel Truck	 	Stormwater
Fueling System	 	Stormwater Infrastructure
Functional Plans	 	Stormwater Pollution Prevention Plan (SWPPP)
Gantry	 	Stormwater Utilities
Garages/Vehicle Maintenance Facilities	 	Strategic Plan
Gas Systems (Propane, Natural, etc.)	 	Streams
General Aviation Airport	 	Substructure
Geographic Information Systems (GIS)	 	SunPass
Geospatial Information Technology (GIT)	 	Superstructure
Girder - Box	 	Surface Mining And Reclamation Act (SMARA)
Girder - Curved	 	Surface Water
Girder - Steel	 	Swimming Pools
GIS Entity Relationship Diagrams (ERD)	 	Tag (Transponder)
GIS Extract Transform Load (ETL)	 	Taxilane
GIS Linear Referencing System (LRS)	 	Taxiway
GIS Metrology	 	Telecommunications - Local Area (LAN) Communication Networks
Global Positioning Systems	 	Telecommunications - Long Haul (Long Distance) Communication Networks
Global Shore Infrastructure Plans	 	Telecommunications - Submarine (Ocean) Communication Networks
Good to Go	 	Telecommunications - Wireless (Cellular, Radio & Microwave) Communications Networks
Government Offices	 	Telephone Systems (Rural/Mobile/Intercom/etc.)
Graphical User Interface (GUI)	 	Threatened/Endangered Species
Halogenated Organics	 	Tie Back System
Hangar/Maintenance Facilities	 	TOD/Station Area Plans
Harbors/Jetties/Piers/Ship Terminal Facilities	 	Toll
Hazard Mitigation Plan (HMP)	 	Toll Booth
Hazardous Waste Phase 1	 	Toll Plaza Facilities
HazMat Spill Contract	 	Toll Plaza Signs
Headquarters Facility	 	Toll Plazas
Heavy Rail Transit	 	Toll System
High Speed Rail/Ground Transportation	 	Toll System - Barrier
High-Rise/Air-Rights-Type Buildings	 	Toll System - Ticket
Highway - Alternatives Analysis	 	Toll System - Video (VTS)
Highway - Arterial	 	TollTag
Highway - Collectors	 	Towers (Self-supporting and Guyed Systems)
Highway - Hard Shoulder Running	 	Toxicology and Hazardous Materials
Highway - HOV	 	Traffic and Transportation
Highway - Interchange	 	Traffic Management Center (TMC)
Highway - Interchange - High-speed	 	Traffic Operations Analysis
Highway - Interchange - Low-speed	 	Traffic Signal
Highway - Interstate/Limited Access	 	Trail
Highway - Non-Limited Access	 	Transponder
Highway - Open Road Tolling	 	Treadle
Highway - Road Diets	 	Truck Parking Facilities
Highway - Rural	 	TSA Basis of Design Report
Highway - Specifications	 	TSA Facilities
Highway - Toll Roads	 	Tunnel
Highway - Urban	 	Tunnels/Subways
Highway - Urban Interchanges	 	Turnpike
Highway Advisory Radio	 	TxTag
Highway Safety Manual	 	Underground Storage Tanks
Highways	 	Uninterruptible Power Supply (UPS)
Historic Districts	 	Unmanned Aerial Vehicle (UAV)
Historic Facility	 	Unmanned Aircraft Systems (UAS)
Historic Monuments and Memorials	 	User Fee Study
Home Program	 	Utilities
Hospital/Medical Facilities	 	Utility Infrastructure
HOT Lanes	 	V-Toll
Hotels/Motels	 	Variable Message Signs (VMS)
Housing	 	Vegetation and Wildlife
Incineration/Landfill	 	Vehicle Separators - Profilers
Incorporation Study	 	Video Processing Center (VPC)
Indefinite Delivery Indefinite Quantity Contracts (IDIQ)	 	Video Processing System (VPS)
Industrial Buildings/Manufacturing Plants	 	Video Production
Industrial Parks	 	Violations Enforcement System (VES)
Infield Area	 	Vision Plan
Infrared Light Curtains	 	Warehouse/Depot
Inspection Report	 	Wastewater
Installation Appearance Plans	 	Water Containment Sources
Installation Development Plans	 	Water Quality Management Plan
Integrated Contingency Response Plans (ICP)	 	Water Quality/TMDL
Integrated Cultural Resource Management Plan (ICRMP)	 	Water Resources and Water Quality
Integrated Natural Resource Management Plan (INRMP)	 	Water Resources/Hydrology/Ground Water
Integrated Product Support Plans	 	Water Supply, Treatment and Distribution Facilities
Intelligent Transportation System (ITS)	 	Water/Wastewater
Inter-Agency Group (IAG)	 	Waterfront Structure
Interchange - Cloverleaf	 	Watershed
Interchange - Diamond	 	Watershed Plans/Master Plan
Interchange - Diverging Diamond	 	Website
Interchange - Partial Cloverleaf	 	Wet Laboratory
Interchange - Rotary	 	Wet Weather
Interchange - Stack	 	Wetland Dependent Bird Species
Interchange - System	 	Wetland Dependent Mammal Species
Interchange - Three-Level Diamond	 	Wetland Dependent Reptilian Species
Interchange - Trumpet	 	Wetlands
Intersection	 	Wild and Scenic Rivers (State/National)
Intersection - Complex	 	Wind Tunnels
Intersection - Turbo Roundabout	 	Wood
Invasive Species	 	Work Zone Safety
Irrigation/Drainage	 	Yard and Terminal
ISO – International Organization for Standardization

# Project Locations

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
path_to_kmz_file = "D:\\MB Projects\\MB Rail Project Locations\\MBI_Rail_Projects_original_no_prior_firms.kmz"

In [ ]:
import zipfile 
import os 
import fiona 
import pandas as pd
import geopandas as gpd 
from pathlib import Path
from bs4 import BeautifulSoup 

In [ ]:
# Specify the directory where we want to extract the KML file (same directory as the KMZ file)
extraction_dir = os.path.dirname("D:\\MB Projects\\MB Rail Project Locations\\MBI_Rail_Projects_original_no_prior_firms.kmz")

# Open the KMZ file and extract its contents
with zipfile.ZipFile(path_to_kmz_file, "r") as kmz:
    kmz.extractall(extraction_dir)

In [ ]:
fiona.drvsupport.supported_drivers['libkml'] = 'rw' 
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'

In [ ]:
kml_path = r"res\all_projects.kml"

baker_projects_df = gpd.GeoDataFrame()

# iterate over layers
for layer in fiona.listlayers(kml_path):
    s = gpd.read_file(kml_path, driver='libkml', layer=layer)
    s['name'] = layer
    baker_projects_df = pd.concat([baker_projects_df, s], ignore_index=True)

In [ ]:
baker_projects_df

In [ ]:
baker_projects_df.to_excel(r"D:\MB Projects\MB Rail Project Locations\Project_Locations.xlsx")

## Project Files

## Pittsburgh Server Files Missing KMZs

In [ ]:
# Keystone West Corridor

## Overall PM

## Hawstone Universal Crossover

# NS Allentown - No kmz file

# NS Harrisburg Auxiliary Lot - No kmz file

# NS Switch Tie Replacement - No kmz file

# Pittsburgh Vertical Clearance


## Great Lakes Server Files

In [ ]:
# Atlanta GA Br. S-293.30 - No proposal kmz.

# Chicago IL Br. BK-306.78 - No proposal kmz.

# Chicago IL Br. CD-509.60 - No proposal kmz.

# Chicago IL Column Repairs - No proposal kmz.

# Cle OH Br. RD-120.94 Superior Ave - No proposal kmz.

# Cle OH Br. RD-121.27 and RD-121.35 - No proposal kmz.

# Cleveland OH Br. RD-119.71 Cedar Ave - No proposal kmz.

# Cleveland OH Br. RD-119.98 - No proposal kmz.

# Cleveland OH Br. RD-120.07 - No proposal kmz.

# Elyria OH Br. CD-206.57 - No proposal kmz.

# Freeburn Ky Wall DB-02 - No proposal kmz.

# Greensburg PA Br. PT 323.85 - No proposal kmz.

# Pittsburgh PA ML-6.04 - No proposal kmz.

# Pittsburgh PA Wall ML-4.8 - No proposal kmz.

# Shelbyville KY Br. 308.50-W - No proposal kmz.

# Suffolk VA Br. N-25.52 - No proposal kmz.

# Welch WV Br. T-1.33 - No proposal kmz.

# Wilkinsburg PA PT Bridges - No proposal kmz.

# Wooster OH Br. PC-135.69 - No proposal kmz.

# NS Emergency Repairs - No proposal kmz.

In [ ]:
# PW Folders that did have KMLs, commented out ones were either NA or already included in main file resulting in duplicates
list_of_files_from_pw = [
    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\Keystone West\NS Altoona_Compiled.kmz",
    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\Keystone West\P-01a Harrisburg-Rockville.kmz",
    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\Keystone West\P-07 Johnstown Rebuild CP C.kmz",
    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\Keystone West\P-03 and P-04 Mifflin Hawstone.kmz",
    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\Keystone West\P-09 Pittsburgh at CP Home.kmz",
    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\Keystone West\P-10 Pittsburgh 3rd Main at Amtrak Station.kmz",
    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\Keystone West\NS PT-353.08.kmz",
    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\Enola 3rd Mainline\NS Enola Project Area.kmz",
    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\NS Johnstown Crossover\NS Johnstown PT-277.6_KMZ.kmz",
    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\NS Lemoyne Connection\Lemoyne Connection.kmz",
    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\NS Johnstown wall at Amtrak Station.kmz",
    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\NS Br. 631.40 Atlanta, GA.kmz",
    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\NS Br. AG-0.1 Chatanooga, TN.kmz",
#    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\NS Br. B-512.06 Stony Island.kmz",
#    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\N-680.20.kmz",
    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\NS Br. N 374.45.kmz",
    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\NS NA-11.1 Wall.kmz",
#    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\NS CD-203.12.kmz",
    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\B-184.50.kmz",
    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\NS PT 275.7 Retaining Wall Project.kmz",
    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\NS H-225.06.kmz",
    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\remington NS Br 56.2.kmz",
#    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\NS S-69.04.kmz",
    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\NS Brdg. 274.00 CNO&TP Roddy, TN.kmz",
    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\NS Br. C-292.20 Sargent, GA.kmz",
#    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\CD-290.93 BoringConcrete Coring.kmz",
    r"C:\Users\dane.parks\Projects\MB Rail Project Locations\PW Projects\NS Br. PT-220.75 Tyrone, PA.kmz"
]

# Other Locations - Railroad Folders, Other Regions, etc?

## Network Drive - Projects

In [ ]:
folder_root = r"J:\Data\Projects\Norfolk Southern"

In [ ]:
network_folder_projects = [
#    r"J:\Data\Projects\Norfolk Southern\156613_NS_Hudson Bridge RD-98.60\RD-98.60.kmz",
#    r"J:\Data\Projects\Norfolk Southern\156613_NS_Hudson Bridge RD-98.60\Working Files\Access to NS RD-98.60.kmz",
#    r"J:\Data\Projects\Norfolk Southern\157860_NS_CD-185.57 & CD-194.36 Steel Repairs\WORKING FILES\Info from NS\CD-185.57\CD-185.57 (NS).kmz",
#    r"J:\Data\Projects\Norfolk Southern\157860_NS_CD-185.57 & CD-194.36 Steel Repairs\WORKING FILES\Info from NS\CD-194.36\CD-194.36 (NS).kmz",
#    r"J:\Data\Projects\Norfolk Southern\162963_NS_CD-185.57 & CD-194.36 Sub Repairs\WORKING FILES\Info From NS\Previous Inspection Reports\CD-185.57\CD-185.57 (NS).kmz",
#    r"J:\Data\Projects\Norfolk Southern\162963_NS_CD-185.57 & CD-194.36 Sub Repairs\WORKING FILES\Info From NS\Previous Inspection Reports\CD-194.36\CD-194.36 (NS).kmz",
#    r"J:\Data\Projects\Norfolk Southern\163299_NS_RD-109.38 Bedford\WORKING FILES\RD-109.38.kmz",
#    r"J:\Data\Projects\Norfolk Southern\170045_NS_CD-185.57 Super Repl_Cleveland OH\400_WORKING FILES\Info from NS\CD-185.57 (NS).kmz",
#    r"J:\Data\Projects\Norfolk Southern\170293_NS_CD-194.36 Super Repl_Berea OH\400_WORKING FILES\Info from NS\Previous Inspection Reports\CD-194.36\CD-194.36 (NS).kmz",
#    r"J:\Data\Projects\Norfolk Southern\170308_NS_CD-487.69 Burn Harbor, IN\400_WORKING FILES\NS Br. CD-487.69.kmz",
#    r"J:\Data\Projects\Norfolk Southern\171228_NS_631.40 Atlanta, GA\400_WORKING FILES\NS Br. 631.40 Atlanta, GA.kmz",
#    r"J:\Data\Projects\Norfolk Southern\171910_NS_Remington VA Br 56.2_Active Files on PW\400_WORKING FILES\430_Environmental Permits\Shapefiles\remington NS Br 56.2.kmz",
#    r"J:\Data\Projects\Norfolk Southern\172045_NS_Kingston N-666.88\N-666.88.kmz",
#    r"J:\Data\Projects\Norfolk Southern\174130_NS_PT-220.75 Tyrone, PA\NS Br. PT-220.75 Tyrone, PA.kmz",
#    r"J:\Data\Projects\Norfolk Southern\175754_NS_B-184.50 Nickel Plate\400_WORKING FILES\NS Br. PT-220.75 Tyrone, PA.kmz",
#    r"J:\Data\Projects\Norfolk Southern\180813_NS_PT-275.7 Wall\400_WORKING FILES\NS Br. PT-220.75 Tyrone, PA.kmz",
#    r"J:\Data\Projects\Norfolk Southern\181422_NS_S-293.30 Atlanta, GA_Active Files on PW\NS Br. S-293.30 Atlanta, GA.kmz",
#    r"J:\Data\Projects\Norfolk Southern\181582_NS_NO-153.53 Pearl River, LA\NS Br. NO-153.53 Pearl River, LA.kmz",
#    r"J:\Data\Projects\Norfolk Southern\184570_NS_C-292.20 Sargent, GA_Active Files on PW\NS Br. C-292.20 Sargent, GA.kmz",
#    r"J:\Data\Projects\Norfolk Southern\185216_NS_CD-487.69 Burns Harbor, IN - CM\600_CONSTRUCTION\630_Field Test Results\Fiber Optic Locates\10421.000 Locate results.kmz",
#    r"J:\Data\Projects\Norfolk Southern\186921_NS_B-115.19 Conneaut OH\NS Br. B-115.19 Conneaut, OH.kmz",
#    r"J:\Data\Projects\Norfolk Southern\186989_NS_B-512.06 Stoney Island_Active Files on PW\NS Br. B-512.06 Stoney Island.kmz",
#    r"J:\Data\Projects\Norfolk Southern\187333_NS_ML-1.85 Pittsburgh, PA\NS Br. ML-1.85 Pittsburgh, PA.kmz",
#    r"J:\Data\Projects\Norfolk Southern\191603_NS_CD-513.86 & CD-514.08 Chicago, IL CM\NS Br. CD-513.86 73rd Street.kmz",
#    r"J:\Data\Projects\Norfolk Southern\191603_NS_CD-513.86 & CD-514.08 Chicago, IL CM\NS Br. CD-514.08 Cottage Grove Ave..kmz",
#    r"J:\Data\Projects\Norfolk Southern\193609_NS-362.60-A Decatur, AL CM\NS Br. 362.60-A Decatur, AL.kmz"
    ]

In [ ]:
temp = ''

for file in network_folder_projects:
    temp = Path(file)
    
    if temp.parent.name[:6].isnumeric():
        print(temp.parent.name[:6])
    elif temp.parent.parent.name[:6].isnumeric():
        print(temp.parent.parent.name[:6])
    elif temp.parent.parent.parent.name[:6].isnumeric():
        print(temp.parent.parent.parent.name[:6])
    elif temp.parent.parent.parent.parent.name[:6].isnumeric():
        print(temp.parent.parent.parent.parent.name[:6])
    elif temp.parent.parent.parent.parent.parent.name[:6].isnumeric():
        print(temp.parent.parent.parent.parent.parent.name[:6])
    else:
        print("If Statement not deep enough")

In [ ]:
temp

## Network Drive - Proposals

In [ ]:
proposal_folder_root = r"J:\Data\Proposal\Norfolk Southern Rail Company"

In [ ]:
list_of_proposal_kmls = [
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2016_151578_Swanton Yard\Proposal\CD-304.14.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2016_XXXXXX_Hudson Bridge\RD-98.60.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2016_XXXXXX_Steel Repairs\CD-185.57 - Madison Avenue\CD-185.57 (NS).kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2016_XXXXXX_Steel Repairs\CD-194.36 - Rocky River Road\CD-194.36 (NS).kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2016_XXXXXX_Steel Repairs\CD-206.93 - East Avenue\CD-206.93 (NS).kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2016_XXXXXX_Steel Repairs\CD-207.26 - West Avenue\CD-207.26 (NS).kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2017_XXXXXX_CD-520.75 & CD-520.91_Chicago\CD-520.75.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2017_XXXXXX_CD-520.75 & CD-520.91_Chicago\CD-520.91.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2017_XXXXXX_CD-520.75 & CD-520.91_Chicago\CD-520.75\doc.kml",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2017_XXXXXX_CD-520.75 & CD-520.91_Chicago\CD-520.91\doc.kml",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2017_XXXXXX_NS Bedford OH\RD-109.38.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2017_XXXXXX_NS Bridge over Cotton Run\NS Bridge CF-35.40 over Cotton Run.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2017_XXXXXX_NS Bridge S-63.12 over Sandusky River\S-63.12.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2018_162365_NS Bridge 56.2 over Tin Pot Run, Remington VA (SELECTED)\NS Br 56.2.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2018_162365_NS Bridge CD-487.69 Burns Harbor, IN (SELECTED)\NS Br. CD-487.69.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2018_166247_NS Bridge B-184.50 Nickel Plate Line (SELECTED)\B-184.50.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2018_166247_NS Bridge B-184.50 Nickel Plate Line (SELECTED)\B-184.50_meeting Place.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2018_166247_NS Bridge B-184.50 Nickel Plate Line (SELECTED)\NS B-184.50 over ODOT Property.kmz",
    ## r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2018_166247_NS Bridge N-399.19 Welch, WV (LOST)\NS Br. N-399.19 Welch, WV.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2019_166247_Bridge N-666.88 Kingston OH (SELECTED)\N-666.88.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2019_166247_Double Stack Clearance Improvements (LOST)\NS Br. R-109.6 Columbia, SC.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2019_166247_Double Stack Clearance Improvements (LOST)\NS Br. R-190.2 Augusta, SC.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2019_166247_NS Bridge 631.4 Atlanta, GA (WON)\NS Br. 631.40 Atlanta, GA.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2019_166247_NS Bridge CD-393.36 Ligonier, IN (SELECTED)\NS Br CD-393.36 Ligonier, IN.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2019_166247_NS Bridge N-680.20 Circleville, OH (SELECTED)\N-680.20.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2019_166247_NS Bridge N-680.20 Circleville, OH (SUPPLEMENT)\N-680.20.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2019_166247_NS Bridge NO-84.50 Hattiesburg, MS (LOST)\NS Br NO-84.50 Hattiesburg, MS.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2019_166247_NS Bridge PT-220.75 Tyrone, PA (SELECTED)\NS Br. PT-220.75 Tyrone, PA.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2020_166247_NS Bridge (3) Greensboro, NC (NO SELECTION)\NS Br. H-39.50 Hillsborough, NC.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2020_166247_NS Bridge H-39.50 Hillsborough, NC (PENDING)\Meeting Location.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2020_166247_NS Bridge H-39.50 Hillsborough, NC (PENDING)\NS Br. H-39.50 Hillsborough, NC.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2020_166247_NS Bridge H-73.60 Rome, GA (LOST)\NS Br H-73.60 Rome, GA.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2020_166247_NS Bridge NO-153.53 Pearl River, LA (WON)\NS Br. NO-153.53 Pearl River, LA.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2020_166247_NS Bridge Ridgeville, SC (PENDING)\NS Ridgeville, SC.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2020_166247_NS Bridge Ridgeville, SC (PENDING)\NS_Ridgeville_ProjectArea.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2020_166247_NS Bridge S-293.30 Atlanta, GA (SELECTED)\NS Br. S-293.30 Atlanta, GA.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2020_166247_NS Bridge S-69.04 Ridgeton, OH (SELECTED)\NS S-69.04.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2020_166247_NS Walls ML-4.8 & PT-275.7 (SELECTED)\ML-4.8\ML-4.8.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2020_166247_NS Walls ML-4.8 & PT-275.7 (SELECTED)\PT-275.7\PT-275.7.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2021_166247_NS Bridge B-512.06 Stoney Island (SELECTED)\NS Br. B-512.06 Stoney Island.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2021_166247_NS Bridge C-292.20 Sargent, GA (SELECTED)\NS Br. C-292.20 Sargent, GA.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2021_166247_NS Bridge CD-290.92 Toledo, OH (SELECTED)\CD-290.93 BoringConcrete Coring.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2021_166247_NS Bridge CD-290.92 Toledo, OH (SELECTED)\CD-290.93.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2021_166247_NS Bridge CD-487.69 Burns Harbor, IN (WON)\NS Br. CD-487.69 Burns Harbor, IN.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2021_166247_NS Bridge H-225.06 Nace, VA (SELECTED)\NS H-225.06.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2021_166247_NS Bridge LC-29.31 Avonmore, PA (LOST)\NS LC-29.31.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2021_166247_NS Bridge ML-1.20 Pittsburgh, PA (LOST)\NS ML-1.2.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2021_166247_NS Bridge ML-1.85 Pittsburgh, PA (SELECTED)\NS Br. ML-1.85 Pittsburgh, PA.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2021_166247_NS Bridge N-655.65 Chillicothe OH (TRACKING)\NS N-655.65.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2021_166247_NS Bridge NO-84.50 Hattiesburg, MS (WON)\NS Br NO-84.50 Hattiesburg, MS.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2021_166247_NS Bridge PC-1.20 Merchant Street CM (LOST)\NS Br. PC-1.20 Merchant Street.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2021_166247_NS Bridge RD-84.96 Ravenna, OH (LOST)\NS RD-98.46.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2021_166247_NS Bridges N-133.41 & N-277.36 Burkeville and Elliston, VA (LOST)\RFP\N 133.41\NS N-133.41.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2021_166247_NS Bridges N-133.41 & N-277.36 Burkeville and Elliston, VA (LOST)\RFP\N 277.36\NS N-277.36.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2021_166247_NS CREATE EW2 Bridges (LOST)\CREATE.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2021_166247_NS Wall DB-0.20 Freeburn, KY (SELECTED)\NS DB-0.2 Wall.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2021_166247_NS Wall NA-11.10 Crum, WV (SELECTED)\NS NA-11.1 Wall.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2022_183873_NS (4) CD Bridges_Amherst OH (LOST)\CD-213.44.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2022_183873_NS (4) CD Bridges_Amherst OH (LOST)\CD-213.53.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2022_183873_NS (4) CD Bridges_Amherst OH (LOST)\CD-213.64.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2022_183873_NS (4) CD Bridges_Amherst OH (LOST)\CD-213.94.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2022_183873_NS Bridge 106.70-A New Market, TN (LOST)\NS Br. 106.70-A New Market, TN.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2022_183873_NS Bridge 316.90 Rathburn, TN (LOST)\NS Br. 316.90 Rathburn, TN.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2022_183873_NS Bridge 328.1-A Brownsboro, AL (LOST)\NS Br. 328.1-A Brownsboro, AL.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2022_183873_NS Bridge 362.60-A Decatur, AL CM (WON)\NS Br. 362.60-A Decatur, AL.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2022_183873_NS Bridge 56.1-H Calhoun, GA (LOST)\NS Br. 56.1-H Calhoun, GA.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2022_183873_NS Bridge AG-3.00 Chatanooga, TN (LOST)\NS Br. AG-3.00 Chattanooga, TN.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2022_183873_NS Bridge BK-306.78 Chicago, IL (SELECTED)\NS Br. BK-306.78 Chicago, IL.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2022_183873_NS Bridge CD-203.12 Elyria OH (SELECTED)\NS CD-203.12.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2022_183873_NS Bridge CD-518.14 45th Street (SELECTED)\NS Br. CD-518.14 Chicago, IL (45th Street).kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2022_183873_NS Bridge ML-6.04 Pittsburgh, PA (WON)\ML-6.04.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2022_183873_NS Bridge RD-119.98 Cleveland OH (SELECTED)\NS RD-119 meeting place.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2022_183873_NS Bridge RD-119.98 Cleveland OH (SELECTED)\NS RD-119.98.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2022_183873_NS Bridge RD-120.07 Cleveland OH (SELECTED)\NS RD-120.07.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2022_183873_NS Bridges CD-513.86 and CD-514.08 Chicago, IL CM (WON)\NS Br. CD-513.86 73rd Street.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2022_183873_NS Bridges CD-513.86 and CD-514.08 Chicago, IL CM (WON)\NS Br. CD-514.08 Cottage Grove Ave..kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2022_183873_NS Dam S-187.50 Macon, GA (NO-GO)\NS Dam S-187.50 Macon, GA.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS (5) CD Bridges_Elyria OH (LOST)\CD-206.82 (NS).kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS (5) CD Bridges_Elyria OH (LOST)\CD-206.93 (NS).kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS (5) CD Bridges_Elyria OH (LOST)\CD-207.26 over West Ave..kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS (5) CD Bridges_Elyria OH (LOST)\CD-207.36 (NS).kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS (5) CD Bridges_Elyria OH (LOST)\NS CD-207.12 over Lake Ave..kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS Br N 374.45 Coopers WV (SELECTED)\NS Br. N 374.45.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS Bridge 168.2-A and 171.6-A Sweetwater, TN (LOST)\NS Br. 168.2-A Sweetwater, TN.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS Bridge 168.2-A and 171.6-A Sweetwater, TN (LOST)\NS Br. 171.6-A Sweetwater, TN.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS Bridge 244.90 Wartburg, TN (LOST)\NS Br. 244.90 Wartburg, TN.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS Bridge 274.00 Roddy, TN (SELECTED)\NS Brdg. 274.00 CNO&TP Roddy, TN.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS Bridge 308.50 W Shelbyville, KY (SELECTED)\NS 308.5 W.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS Bridge 332.50 CNO&TP Chattanooga, TN (LOST)\NS Br. 332.50 CNO&TP Chatanooga, TN.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS Bridge AG-0.10 Chatanooga, TN (WON)\NS Br. AG-0.1 Chatanooga, TN.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS Bridge CD-206.57 over East Bridge Street Elyria, OH (SELECTED)\CD-206.57 (NS).kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS Bridge CD-206.57 over East Bridge Street Elyria, OH (SELECTED)\Proposal\Geotech - NEAS\206.57 Borings.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS Bridge PC-135.69 Wooster, OH (SELECTED)\NS PC-135.69 Wooster OH.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS Bridge PT-323.86 Greensburg, PA (SELECTED)\NS PT-323.86.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS Bridge RD-119.71 Cleveland OH (SELECTED)\NS RD-119.71.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS Bridge RD-120.94 Superior Ave_Cleveland OH (SELECTED)\NS RD-120.94.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS Bridge RD-121.27 over Hamilton Ave_RD-121.35 over E 33rd_Cleveland OH (SELECTED)\NS RD-121.27.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS Bridge RD-121.27 over Hamilton Ave_RD-121.35 over E 33rd_Cleveland OH (SELECTED)\NS RD-121.35.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS Bridge RD-84.96 Ravenna, OH (LOST)\NS RD-98.46.kmz",
    r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS Bridge T-1.33 Welch WV (SELECTED)\NS Welch, WV.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2023_183873_NS Tunnel P-421.90 Leeds, AL (NO-GO)\NS Tunnel P-421.90 Leeds, AL.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2024_183873_NS (3) Bridges PT-346 Wilkinsburg, PA\NS PT-346.27.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2024_183873_NS (3) Bridges PT-346 Wilkinsburg, PA\NS PT-346.28.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2024_183873_NS (3) Bridges PT-346 Wilkinsburg, PA\NS PT-346.41.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2024_183873_NS Bridge 362.60-A Decatur AL (PENDING)\NS Br. 362.60-A Decatur, AL.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2024_183873_NS Bridge 88.2-MB Jackson AL (PENDING)\NS Br. 88.20-MB Jackson, AL.kmz",
    # r"J:\Data\Proposal\Norfolk Southern Rail Company\TRN_2024_183873_NS Bridge PT-353.08 Pgh Amtrak Station (PENDING)\NS PT-353.08.kmz"
]

In [ ]:
csx_proposal_root = r"J:\Data\Proposal\CSX Transportation"

In [ ]:
list_of_csx_proposals = [
    r"J:\Data\Proposal\CSX Transportation\2021_STR_XXXXXX_Bridge QR 110.24 Rating\CSX - QR 110.24.kmz",
    r"J:\Data\Proposal\CSX Transportation\2022_STR_000 583.10\CSX 000 583.10.kmz",
    r"J:\Data\Proposal\CSX Transportation\2022_STR_XXXXXX_Br 00H 313.8 Rating\CXS Bridge 00H_315.00.kmz",
    r"J:\Data\Proposal\CSX Transportation\2022_STR_XXXXXX_CJ 21.60\CSX CJ 21.60.kmz",
    r"J:\Data\Proposal\CSX Transportation\2022_STR_XXXXXX_CLV 0.10 2023 TSC Survey\CSX CLV 0.10.kmz",
    r"J:\Data\Proposal\CSX Transportation\2022_STR_XXXXXX_QB 146.34\CSX QB 146.34.kmz",
    r"J:\Data\Proposal\CSX Transportation\2024_STR_195341_Br 00K 623.5 Pine Barren, FL\CSX Br 00K 623.5.kmz",
    r"J:\Data\Proposal\CSX Transportation\2024_STR_195341_Toledo Coal Dock Wall\CSX Toledo Coal Dock Wall.kmz"
]

In [ ]:
df_dict_2 = {}

combined_folder_list = network_folder_projects + list_of_proposal_kmls + list_of_csx_proposals

for i, file in enumerate(combined_folder_list):
    kml_path = Path(file)
    print(kml_path)
    
    # iterate over layers
    for layer in fiona.listlayers(kml_path):
        s = gpd.read_file(kml_path, driver='libkml', layer=layer)
        s['name'] = layer
        df_dict_2[i] = s

In [ ]:
# Getting rid of boring logs and other kml file types (they have more columns than std. maps)
for key, value in df_dict_2.items():
    print(f"{key}, {len(value.columns)}")

In [ ]:
del df_dict_2[13]
del df_dict_2[9]

In [ ]:
merged_data2 = pd.concat(df_dict_2, ignore_index=True)

In [ ]:
merged_data2

In [ ]:
all_data = pd.concat([merged_data1, merged_data2, baker_projects_df], ignore_index=True)

In [ ]:
all_data.to_excel('output/all_data.xlsx')

In [ ]:
import shapely
import simplekml

# Remove Rows that are lines instead of points
for index, row in all_data.iterrows():
    if type(row['geometry']) == shapely.geometry.point.Point:
        pass
    else:
        all_data = all_data.drop([index])

In [ ]:
kml = simplekml.Kml(open=1)

In [ ]:
for index, row in all_data.iterrows():
    pnt = kml.newpoint()
    pnt.name = row['Name']
    pnt.description=row['description']
    pnt.coords = [(row['geometry'].x, row['geometry'].y)]

kml.save('output/all_data.kml')

In [ ]:
gpd.datasets.available

In [ ]:
# Get Ohio boundary
us_states_shp_file = r"C:\Users\dane.parks\PycharmProjects\civilpy\Notebooks\res\tl_2023_us_state.shp"

all_states = gpd.read_file(us_states_shp_file)
ohio = all_states.iloc[[24]]

In [ ]:
ohio.plot()

In [ ]:
filtered_dataframe = all_data.within(ohio.loc[24, 'geometry'])

In [ ]:
ohio_projects = all_data.loc[filtered_dataframe].copy()

In [ ]:
len(ohio_projects)

In [ ]:
# Drop duplicate points
ohio_projects = ohio_projects.drop_duplicates(subset=['geometry'])

In [ ]:
len(ohio_projects)

In [ ]:
kml2 = simplekml.Kml(open=1)

# Save the values to kml
for index, row in ohio_projects.iterrows():
    pnt = kml2.newpoint()
    pnt.name = row['Name']
    pnt.description=row['description']
    pnt.coords = [(row['geometry'].x, row['geometry'].y)]

kml2.save('output/ohio_projects.kml')

In [ ]:
ohio_projects.to_excel('output/ohio_data.xlsx')

In [ ]:
ohio_projects

# Cleveland File Server Directory Search

In [ ]:
# Add Folders to be included in search here;
project_list = next(os.walk("L:\\Projects\\Norfolk Southern"))[1]

In [ ]:
# This cell is the search function
import re
r = re.compile(".*59th.*")  # This is the regex string to search against
filtered_list = list(filter(r.match, project_list)) # Read Note below

In [ ]:
# This cell displays results
filtered_list

## Link to Working Excel File

https://mbakerintl-my.sharepoint.com/:x:/r/personal/dane_parks_mbakerintl_com/Documents/Projects/allfiles.xlsx?d=w67e96dc3ac03408aa84644e647e53ea5&csf=1&web=1&e=pq35jT

# Baker Office Shape Files

In [ ]:
# Import necessary modules
import geopandas as gpd

# Set filepath
fp = "res/MB Offices/Michael_Baker_International_Offices.shp"

# Read file using gpd.read_file()
data = gpd.read_file(fp)

In [ ]:
data

In [ ]:
data.geometry.total_bounds

In [ ]:
import matplotlib.pyplot as plt

f, ax = plt.subplots()
ax.set_xlim(-17572922.1052, -7934766.7022)
ax.set_ylim(2428716.6456, 9559028.21660598)
plt = data.plot()


# Static Map Example

In [ ]:
import contextily as cx

ax = data.plot(figsize=(20, 20), alpha=0.5, markersize=100, edgecolor="k")
cx.add_basemap(ax, zoom=4, crs=data.crs)

# Interactive Map Example

In [ ]:
# values = json.loads(search_results)

In [ ]:
df = pd.DataFrame(search_results)

In [ ]:
test_user = search_results[0]

In [ ]:
for value in test_user:
    print(value)

In [ ]:
dict(test_user)

In [ ]:
for value in test_user:
    print(f"{value}: {test_user[value]}")

In [ ]:
list_of_users = []
multi_value_list = [
    'FlattenedLicensesCertifications', 
    'FlattenedSkills', 
    'FlattenedComputerSkills', 
    'Skills', 
    'Education', 
    'ComputerSkills', 
    'ProfessionalAffiliations', 
    'LicensesCertifications'
]

for index, employee in enumerate(search_results):
    print(index, employee['FirstName'], employee['LastName'])
    
    for value in employee:
        if value in multi_value_list:
            for value_2 in search_results[index][value]:
                print(f"\t\t{value_2}: {search_results[index][value][value_2]}")
        else:
            print(f"\t{value}: {search_results[index][value]}")

In [ ]:
gdf = pd.DataFrame(list_of_users)

In [ ]:
gdf.head(10)

In [ ]:
from geopy.geocoders import Nominatim

# Build Employee GeoDataFrame
loc = Nominatim(user_agent="GetLoc")

getLoc = loc.geocode(search_results[0].BakerOffice)

getLoc.latitude, getLoc.longitude

In [ ]:
import folium
data.explore(
    marker_kwds=dict(
        radius=8, 
        fill=True, 
        icon=folium.map.Icon(
            color='black',
            prefix='fa',
            icon='building'
        )
    ), 
    marker_type='marker'
)

In [ ]:
for Employee in search_results[:10]:
    print(f"Name: {Employee.FirstName} {Employee.LastName}")
    print(f"Headquarters: {Employee.BakerOffice}")
    print(f"Region: {Employee.Region}")
    print(f"Department: {Employee.Department}\n\n")